In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from itertools import product

# --- funções de métricas hidrológicas ---
def rmse(observed, simulated):
    return np.sqrt(mean_squared_error(observed, simulated))

def nse(observed, simulated):
    """Nash–Sutcliffe Efficiency"""
    obs = np.array(observed)
    sim = np.array(simulated)
    return 1 - np.sum((obs - sim)**2) / np.sum((obs - np.mean(obs))**2)

def kge(observed, simulated):
    """Kling-Gupta Efficiency"""
    obs = np.array(observed)
    sim = np.array(simulated)
    r = np.corrcoef(obs, sim)[0,1]
    alpha = np.std(sim) / np.std(obs)
    beta  = np.mean(sim) / np.mean(obs)
    return 1 - np.sqrt((r-1)**2 + (alpha-1)**2 + (beta-1)**2)

def pbias(observed, simulated):
    """Percent Bias"""
    return 100.0 * np.sum(simulated - observed) / np.sum(observed)

# --- prepara features e target ---
def prepare_features(df, n_lags, n_ahead, use_monthly_mean):
    df = df.copy().sort_values('Data').reset_index(drop=True)
    df['month'] = df['Data'].dt.month

    # 1) dessazonalizar:
    if use_monthly_mean:
        split_idx = int(len(df) * 0.7)
        train_base = df.iloc[:split_idx]
        mens = train_base.groupby('month')['Vazao'].mean()
        df['Vazao'] = df['Vazao'] - df['month'].map(mens)

    # 2) gera lags e target a partir da série já dessazonalizada
    for lag in range(1, n_lags+1):
        df[f'lag_{lag}'] = df['Vazao'].shift(lag-1)
    df[f'y_{n_ahead}d'] = df['Vazao'].shift(-n_ahead)

    # 3) descarta NA, divide em treino/teste e extrai X/y
    cols_X = [f'lag_{lag}' for lag in range(1, n_lags+1)]
    df = df.dropna(subset=cols_X + [f'y_{n_ahead}d']).reset_index(drop=True)
    split_idx = int(len(df) * 0.7)
    df_train, df_test = df.iloc[:split_idx], df.iloc[split_idx:]

    X_train = df_train[cols_X].values
    y_train = df_train[f'y_{n_ahead}d'].values
    X_test  = df_test[cols_X].values
    y_test  = df_test[f'y_{n_ahead}d'].values

    # se precisar do vetor de média para reverter só no teste:
    vaz_med_test = None
    if use_monthly_mean:
        vaz_med_test = df_test['month'].map(mens).values

    return X_train, X_test, y_train, y_test, vaz_med_test




# --- loop de experimentos ---
def run_experiments(df):
    results = []
    opts_vaz = ['Sim', 'Não'] # = ['Sim', 'Não']
    opts_lags = [1, 7, 30] # = [1, 7, 30]
    opts_ahead = [1, 2, 7, 30] # = [1, 2, 7, 30]

    #callback do early stopping antes do loop (ou logo antes de treinar cada modelo)
    early_stop = tf.keras.callbacks.EarlyStopping(
        monitor='val_loss',
        patience=200,
        min_delta=1e-4,
        restore_best_weights=True)

    for vaz_opt, n_lags, n_ahead in product(opts_vaz, opts_lags, opts_ahead):
        use_mm = (vaz_opt == 'Sim')

        X_train, X_test, y_train, y_test, vaz_med = prepare_features(df, n_lags, n_ahead, use_mm)
        print(f"\n>>> n_ahead = {n_ahead}")
        print("y_test head:", y_test[:5])
        print("Primeiras 5 linhas de X_test:\n", X_test[:5, :])
        # escalonamento
        scaler = MinMaxScaler()
        X_train = scaler.fit_transform(X_train)
        X_test  = scaler.transform(X_test)

        # monta o modelo keras
        model = tf.keras.Sequential([
            tf.keras.layers.InputLayer(input_shape=(X_train.shape[1],)),
            tf.keras.layers.Dense(128, activation='relu'),
            tf.keras.layers.Dense(64, activation='relu'),
            tf.keras.layers.Dense(1)
        ])
        model.compile(optimizer='adam', loss='mse')

        #treina
        history = model.fit(
            X_train, y_train,
            epochs=625,
            batch_size=32,
            validation_split=0.2,
            callbacks=[early_stop],
            verbose=0
        )

        # extrai quantas épocas rodaram
        epochs_by_history = len(history.history['loss'])

        #previsao
        y_pred = model.predict(X_test).flatten()

        if use_mm== True:
            y_pred= y_pred+vaz_med
            y_test= y_test+vaz_med

        # métricas
        rmse_  = rmse(y_test, y_pred)
        nse_   = nse(y_test, y_pred)
        kge_   = kge(y_test, y_pred)
        pbias_ = pbias(y_test, y_pred)

        results.append({
            'vazao_menos_vaz_mensal_med': vaz_opt,
            'n_vazoes_anteriores':      n_lags,
            'n_dias_a_frente':          n_ahead,
            'RMSE':  rmse_,
            'NSE':   nse_,
            'KGE':   kge_,
            'Pbias': pbias_,
            'epochs':epochs_by_history
        })

        print(f'Concluído: mensal_med={vaz_opt}, lags={n_lags}, ahead={n_ahead}')

    return pd.DataFrame(results)

if __name__ == '__main__':
    #ler a série histórica
    #df = pd.read_csv('uniaoh.csv', parse_dates=['Data'])
    df = pd.read_csv('uniaoh.csv')
    df['Data'] = pd.to_datetime(df['Data'], format='%d/%m/%Y')

    resultados = run_experiments(df)
    resultados.to_csv('resultados_metricas.csv', index=False)
    print('Todos os cenários executados.')



>>> n_ahead = 1
y_test head: [-236.24216126 -254.44116126 -263.42516126 -260.43916126 -248.40916126]
Primeiras 5 linhas de X_test:
 [[-223.94416126]
 [-236.24216126]
 [-254.44116126]
 [-263.42516126]
 [-260.43916126]]


c:\Users\PersioPuertasGarciaL\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\core\input_layer.py:27: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


297/297 ━━━━━━━━━━━━━━━━━━━━ 0s 467us/step
Concluído: mensal_med=Sim, lags=1, ahead=1

>>> n_ahead = 2
y_test head: [-236.24216126 -254.44116126 -263.42516126 -260.43916126 -248.40916126]
Primeiras 5 linhas de X_test:
 [[-202.11216126]
 [-223.94416126]
 [-236.24216126]
 [-254.44116126]
 [-263.42516126]]


c:\Users\PersioPuertasGarciaL\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\core\input_layer.py:27: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


297/297 ━━━━━━━━━━━━━━━━━━━━ 0s 610us/step
Concluído: mensal_med=Sim, lags=1, ahead=2

>>> n_ahead = 7
y_test head: [-263.42516126 -260.43916126 -248.40916126 -263.42516126 -278.22316126]
Primeiras 5 linhas de X_test:
 [[-160.57416126]
 [-157.32716126]
 [-176.69916126]
 [-202.11216126]
 [-223.94416126]]


...

296/296 ━━━━━━━━━━━━━━━━━━━━ 0s 493us/step
Concluído: mensal_med=Não, lags=30, ahead=2

>>> n_ahead = 7
y_test head: [173.367 170.589 173.367 162.32  159.585]
Primeiras 5 linhas de X_test:
 [[ 236.932  233.946  242.93   261.129  273.427  295.259  320.672  340.044
   336.797  369.584  416.63   493.08   493.08   510.86   605.58   723.141
   773.832  830.018  887.743  937.976  993.158 1023.193 1049.112 1079.554
  1083.92  1066.481 1062.132 1079.554 1101.429 1110.21 ]
 [ 248.962  236.932  233.946  242.93   261.129  273.427  295.259  320.672
   340.044  336.797  369.584  416.63   493.08   493.08   510.86   605.58
   723.141  773.832  830.018  887.743  937.976  993.158 1023.193 1049.112
  1079.554 1083.92  1066.481 1062.132 1079.554 1101.429]
 [ 233.946  248.962  236.932  233.946  242.93   261.129  273.427  295.259
   320.672  340.044  336.797  369.584  416.63   493.08   493.08   510.86
   605.58   723.141  773.832  830.018  887.743  937.976  993.158 1023.193
  1049.112 1079.554 1083.92  106

c:\Users\PersioPuertasGarciaL\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\core\input_layer.py:27: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


296/296 ━━━━━━━━━━━━━━━━━━━━ 0s 488us/step
Concluído: mensal_med=Não, lags=30, ahead=7

>>> n_ahead = 30
y_test head: [173.367 228.    245.942 236.932 416.63 ]
Primeiras 5 linhas de X_test:
 [[ 773.832  830.018  887.743  937.976  993.158 1023.193 1049.112 1079.554
  1083.92  1066.481 1062.132 1079.554 1101.429 1110.21   963.347  711.557
   521.597  148.753  132.844  154.147  165.066  184.578  195.949  222.09
   239.926  251.991  258.075  255.029  245.942  242.93 ]
 [ 723.141  773.832  830.018  887.743  937.976  993.158 1023.193 1049.112
  1079.554 1083.92  1066.481 1062.132 1079.554 1101.429 1110.21   963.347
   711.557  521.597  148.753  132.844  154.147  165.066  184.578  195.949
   222.09   239.926  251.991  258.075  255.029  245.942]
 [ 605.58   723.141  773.832  830.018  887.743  937.976  993.158 1023.193
  1049.112 1079.554 1083.92  1066.481 1062.132 1079.554 1101.429 1110.21
   963.347  711.557  521.597  148.753  132.844  154.147  165.066  184.578
   195.949  222.09   239.926  2

c:\Users\PersioPuertasGarciaL\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\core\input_layer.py:27: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


296/296 ━━━━━━━━━━━━━━━━━━━━ 0s 488us/step
Concluído: mensal_med=Não, lags=30, ahead=30
Todos os cenários executados.


In [22]:
resultados

,vazao_menos_vaz_mensal_med,n_vazoes_anteriores,n_dias_a_frente,RMSE,NSE,KGE,Pbias,epochs
0,Sim,1,1,67.109121,0.981349,0.989799,0.416217,625
1,Sim,1,2,120.054436,0.940351,0.966351,-0.555785,625
2,Sim,1,7,293.365218,0.644752,0.748228,-3.037348,625
3,Sim,1,30,475.185130,0.050721,0.028165,-15.106114,318
4,Sim,7,1,52.781198,0.988464,0.988619,-0.631119,323
5,Sim,7,2,99.791296,0.958791,0.968594,-0.852537,529
6,Sim,7,7,267.304225,0.705093,0.797446,-2.817022,625
7,Sim,7,30,474.592151,0.053100,0.040523,-13.590626,576
8,Sim,30,1,57.876095,0.986134,0.985712,-0.480243,297
9,Sim,30,2,101.154919,0.957670,0.971080,-0.288789,329
